In [0]:
import requests 
from pyspark.sql import DataFrame

In [0]:
with requests.get('https://datahub.io/core/glacier-mass-balance/r/glaciers.csv', stream=True) as r:
    with open('/dbfs/glacier.csv', 'wb') as f:
        for chunk in r.iter_content(chunk_size = 8192):
            f.write(chunk)

In [0]:
def get_data(url:str):
    filename = url.split('/')[-1]
    with requests.get('https://datahub.io/core/glacier-mass-balance/r/glaciers.csv', stream=True) as r:
        with open('/dbfs/glacier.csv', 'wb') as f:
            for chunk in r.iter_content(chunk_size = 8192):
                f.write(chunk)
    return filename

In [0]:
data = spark.read.format('csv').option('header', 'true').load('file:/dbfs/glacier.csv')

In [0]:
data.show(2)

+----+----------------------------+----------------------+
|Year|Mean cumulative mass balance|Number of observations|
+----+----------------------------+----------------------+
|1945|                           0|                  null|
|1946|                       -1.13|                     1|
+----+----------------------------+----------------------+
only showing top 2 rows



In [0]:
data.createOrReplaceTempView('newdf')

In [0]:
%sql select * from newdf

Year,Mean cumulative mass balance,Number of observations
1945,0,null
1946,-1.13,1
1947,-3.19,1
1948,-3.19,1
1949,-3.82,3
1950,-4.887,3
1951,-5.217,3
1952,-5.707,3
1953,-6.341,7
1954,-6.825,6


In [0]:
%sql 
create or replace temp view ninties as select * from newdf where Year like '19%' order by Year asc;
create or replace temp view modern as select * from newdf where Year like '20%' order by Year asc;

In [0]:
nineties_df = spark.sql('select * from ninties')
modern_df = spark.sql('select * from modern')

In [0]:
x, y = nineties_df, modern_df

In [0]:
display(x)

Year,Mean cumulative mass balance,Number of observations
1945,0,null
1946,-1.13,1
1947,-3.19,1
1948,-3.19,1
1949,-3.82,3
1950,-4.887,3
1951,-5.217,3
1952,-5.707,3
1953,-6.341,7
1954,-6.825,6


In [0]:
display(y)

Year,Mean cumulative mass balance,Number of observations
2000,-17.727,37
2001,-18.032,37
2002,-18.726,37
2003,-19.984,37
2004,-20.703,37
2005,-21.405,37
2006,-22.595,37
2007,-23.255,37
2008,-23.776,37
2009,-24.459,37


In [0]:
nineties_file_name = spark.sql('(select * from ninties order by Year asc limit 1) union (select * from ninties order by Year desc limit 1)')
modern_file_name = spark.sql('(select * from modern order by Year asc limit 1) union (select * from modern order by Year desc limit 1)')

In [0]:
display(nineties_file_name)

Year,Mean cumulative mass balance,Number of observations
1945,0,null
1999,-17.697,37


In [0]:
display(modern_file_name)

Year,Mean cumulative mass balance,Number of observations
2000,-17.727,37
2014,-28.652,24


In [0]:
modern_file_name_df = modern_file_name.collect()
nineties_file_name_df = nineties_file_name.collect()

In [0]:
modern_file_name = modern_file_name_df[0].__getitem__('Year') + '-' + modern_file_name_df[1].__getitem__('Year')
nineties_file_name = nineties_file_name_df[0].__getitem__('Year') + '-' + nineties_file_name_df[1].__getitem__('Year')

In [0]:
nineties_df.write.format('parquet').save('/dbfs/nineties_df.parquet')
modern_df.write.format('parquet').save('/dbfs/modern_df.parquet')

In [0]:
m, n = nineties_df, modern_df

In [0]:
def write_df(filetype:str, dfs, filenames):
    for x, y in zip(dfs, filenames):
        m = x.write.format(filetype).save('/dbfs/{}.{}'.format(y, filetype))

In [0]:
write_df('parquet', [x, y], [m, n])

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1870112663028074>:1
----> 1 write_df('parquet', [x, y], [m, n])

File <command-1870112663028073>:3, in write_df(filetype, dfs, filenames)
      1 def write_df(filetype:str, dfs, filenames):
      2     for x, y in zip(dfs, filenames):
----> 3         m = x.write.format(filetype).save('/dbfs/{}.{}'.format(y, filetype))

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1397, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1

In [0]:
dbutils.fs.ls('/dbfs/')

Out[67]: [FileInfo(path='dbfs:/dbfs/DataFrame[Year: string, Mean cumulative mass balance: string, Number of observations: string].parquet/', name='DataFrame[Year: string, Mean cumulative mass balance: string, Number of observations: string].parquet/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/dbfs/modern_df.parquet/', name='modern_df.parquet/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/dbfs/nineties_df.parquet/', name='nineties_df.parquet/', size=0, modificationTime=0)]